In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 
import cv2
warnings.filterwarnings("ignore")
import random

ModuleNotFoundError: No module named 'google'

## 0. 목차 기준 Titanic Data EDA, Titanic Survival Modeling, Cross Validation 코드를 전부 필사하라.

In [ ]:
titanic = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/titanic.csv")
titanic.head()

In [ ]:
titanic.shape #데이터 891개, 컬럼 12개

In [ ]:
titanic.info() #Name, Sex, Ticket, Cabin, Embarked는 문자형으로 되어있음

In [ ]:
titanic.describe()

In [ ]:
#결측치 확인
titanic.isnull().sum()

In [ ]:
#Age, Cabin, Embarked에서 결측치가 발견되었는데 어떻게 처리할까?
#Age는 평균으로 처리
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].mean())

In [ ]:
#cabin은 결측치가 거의 대부분인데 필요할까? 이 피처는 쓰지 않는게 좋겠음
titanic = titanic.drop(["Cabin"], axis = 1)
titanic.head()

In [ ]:
#Embarked는 어디에서 승선했는지의 여부인데 categorical variables이므로 최빈값으로 대체
#결측치가 2개뿐이므로 처리방식은 그다지 중요하지 않을듯
titanic["Embarked"].value_counts()

In [ ]:
titanic["Embarked"] = titanic["Embarked"].fillna(titanic["Embarked"].value_counts().index[0])

In [ ]:
titanic.isna().sum()

In [ ]:
#target 접근 : 얼마나 많은 사람들이 살아남았는가?

In [ ]:
ax = plt.figure(figsize = (16, 5)) #빈 캔버스 생성
plt.pie(titanic["Survived"].value_counts(), explode = [0.01, 0.01], labels = ["No", "Yes"], autopct = "%.2f")
#생존자의 수를 바탕으로 파이플롯 작성. explode는 각 파이들이 얼마나 중점으로부터 떨어질지를 결정. labels는 각 파이의 이름을 의미. autopct는 자동으로 퍼센트를 표기하겠다는 의미.
plt.title("Pie plot of the people survived")
plt.show()

In [ ]:
#성별과 생존의 상관관계 분석
titanic.groupby(by = ["Sex", "Survived"])["Survived"].count()
#남성이 여성보다 많이 죽은 것을 확인

In [ ]:
#Sex, Survived모두 categorical variables이기 때문에 catplot이 적절함
sns.catplot(x = "Survived", kind = "count", hue = "Sex", data = titanic)
plt.title("Sex-Survival?")
plt.show()

In [ ]:
#티켓 클래스와 생존간의 상관관계 분석
#티켓 클래스는 categorical 변수 중에서도 순서형 변수 (ordinal variable) - 그 숫자의 순위가 의미를 갖고 있음. 1은 최우등석, 2는 우등석, 3은 일반석
sns.barplot(x = titanic["Pclass"].value_counts().index, y = titanic["Pclass"].value_counts())

In [ ]:
sns.catplot(x = "Pclass", kind = "count", hue = "Survived", data = titanic)
#아무래도 1등석에 탄 사람들이 생존 확률이 높았음 - 구조 확률이 더 높았을 듯

In [ ]:
#feature Engineering
#sibsp, parch는 각각 친척/배우자의 수와 부모님 수를 의미함, 이를 합쳐서 family_size라는 변수로 만들어보자
#이와 동시에 sibsp+parch인 총 동승 가족의 수가 0명인 사람에게 alone이라는 변수명을 부여

In [ ]:
titanic['Family_Size'] = 0 #Family_Size라는 열 생성 및 0으로 초기화
titanic['Family_Size'] = titanic['Parch'] + titanic['SibSp'] #가족 수 계산
titanic['Alone'] = 0 #Alone이라는 이진분류용 열 생성 및 0으로 초기화
titanic.loc[titanic["Family_Size"] == 0, "Alone"] = 1 #family_size가 0인 사람은 1로 라벨링, 나머지는 여전히 0 

In [ ]:
#Name, PassengerID, Ticket은 중요하지 않은 변수 - 생존분석에 어떠한 도움도 주지 않는다는 것을 직관적으로 이해할 수 있음.
titanic = titanic.drop(["Name", "PassengerId", "Ticket"], axis = 1)
titanic.head()

In [ ]:
#Sex, Embarked 변수는 아직 categorical variable 상태이므로 인코딩 - Label Encoding
from sklearn.preprocessing import LabelEncoder

le_sex = LabelEncoder()
le_sex.fit(titanic["Sex"])
titanic["Sex"] = le_sex.transform(titanic["Sex"])
titanic.head()
#Male = 1, Female = 0으로 인코딩

In [ ]:
le_embarked = LabelEncoder()
le_embarked.fit(titanic["Embarked"])
titanic["Embarked"] = le_embarked.transform(titanic["Embarked"])
titanic.head()

In [ ]:
#titanic Survival Modeling
#필요 라이브러리 임포트
#모델
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from xgboost import XGBClassifier #XGBoost
import lightgbm as lgb #LightGBM
from sklearn import tree

In [ ]:
#기타
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
import graphviz 

In [ ]:
X = titanic.drop(["Survived"], axis = 1)
y = titanic["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 156)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#712개의 데이터가 training, 179개 데이터가 test에서 쓰임 

In [ ]:
#어떤 모델이든 그 성능을 여러 지표로 평가해 보여주는 함수를 제작
def evaluate_metrics(model, X_test, y_test):
  y_hat = model.predict(X_test)
  print("Confusion matrix: ", confusion_matrix(y_test, y_hat), "\n")
  print("Accuracy: ", accuracy_score(y_test, y_hat), "\n")
  print("Precision: ", precision_score(y_test, y_hat), "Recall: ", recall_score(y_test, y_hat), "\n")
  print("F1 score: ", f1_score(y_test, y_hat), "\n")

In [ ]:
#Logistic Regression
Log_R = LogisticRegression()
Log_R.fit(X_train, y_train)
evaluate_metrics(Log_R, X_test, y_test)

In [ ]:
#Support Vector Machine 학습
svc = svm.SVC(kernel = "linear", C = 0.1, gamma = 0.1)
svc.fit(X_train, y_train)
evaluate_metrics(svc, X_test, y_test)

In [ ]:
#Decision Tree 학습
DTclf = DecisionTreeClassifier(max_depth = 4)
DTclf.fit(X_train, y_train)
evaluate_metrics(DTclf, X_test, y_test)

In [ ]:
X.head()

In [ ]:
#DecisionTree 다른 방식으로 시각화
dot_data4 = tree.export_graphviz(DTclf, out_file = None,
                                 filled = True, rounded = True,
                                 special_characters = True, impurity = False, class_names= ["Survived", "Dead"]) #decision tree 시각화
graph4 = graphviz.Source(dot_data4)
graph4

In [ ]:
#RandomForest 학습
RfClf = RandomForestClassifier(n_estimators = 100, max_depth = 4) #n_estimators라는 것은 숲속 나무 개수, 즉 각기 다른 tree 모델들의 개수 
RfClf.fit(X_train, y_train)
evaluate_metrics(RfClf, X_test, y_test) #확실히 성능이 좋음 - Decision Tree가 100개 이므로

In [ ]:
#LightGBM 학습
lgbClf = lgb.LGBMClassifier(num_leaves = 20)
lgbClf.fit(X_train, y_train)
evaluate_metrics(lgbClf, X_test, y_test)

In [ ]:
#Cross validation
from sklearn.model_selection import KFold
X = np.array(titanic.drop(["Survived"], axis = 1)) #array로 설정해주는 것은 컴파일 에러를 막기 위해
y = titanic["Survived"] 

#n_splits는 몇개의 부분집합으로 분할할 것인지를 나타냄 - 5개로 나누었으니 학습 다섯 차례 진행
kf = KFold(n_splits = 5, shuffle = True, random_state = 50)

#split 개수 스텝만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in kf.split(X):
  print("검증 데이터 인덱스:", test_index)

"""
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
"""

In [ ]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 50)

accuracy_history = []
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  rfClf = RandomForestClassifier(n_estimators = 5, random_state = 0) #예시로 RandomForest모델 활용
  rfClf.fit(X_train, y_train)
  y_hat = rfClf.predict(X_test)
  accuracy_history.append(accuracy_score(y_hat, y_test))

print("평균 점수: ", np.mean(accuracy_history))

In [ ]:
#위의 과정을 조금 더 단순하게 할 수 있는 코드 - 윗 셀보다 이 코드에 더 집중하자
from sklearn.model_selection import cross_val_score #라이브러리 임포트
kf = KFold(n_splits = 5, shuffle = True, random_state = 50) #KFold 객체 생성

rfClf = RandomForestClassifier(n_estimators= 5, random_state = 0) #예시로 random forest 이용

cv_result = cross_val_score(rfClf, X, y, cv=kf, scoring = "accuracy")
print(cv_result.mean())

## 1. bank.csv 를 다운로드 받고 info, describe등을 활용해 데이터의 개요를 파악하라.

In [ ]:
bank = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/bank.csv")

In [ ]:
bank.info()

In [ ]:
bank.describe()

In [ ]:
bank.head()

In [ ]:
bank.isnull().sum() #결측치 없는 데이터(!)

## 2. 데이터에 대해 각기 다른 방식으로 총 3가지 시각화를 수행하라.

In [ ]:
#age-balance 사이의 관계 시각화
#둘다 수치형 변수
sns.lineplot(x = 'age', y='balance', data = bank)
plt.show()

In [ ]:
#loan 여부에 따른 balance 분포
plt.boxplot([bank.loc[bank["loan"]=="yes", "balance"], bank.loc[bank["loan"]=="no", "balance"]])
plt.xticks([1, 2], ["yes", "no"])
plt.show()
#생각보다 outlier가 많음... 별로 의미 있는 분석은 아닌 듯

In [ ]:
#job에 따른 평균 나이
df = bank.groupby(by = ["job"]).mean()
plt.barh(df.index, df["age"])
plt.show()
#은퇴한 사람은 확실히 나이가 많다. 학생은 어리다. 나머지는 다 비슷비슷하게 중년나이인듯.

## 3. 컬럼 간 연산을 통해 분석에 필요할 것이라고 생각하는 컬럼을 추가하고 근거를 제시하라.

In [ ]:
#default, housing, loan -> 모두 대출... 이런쪽 -> 하나로 합쳐서 표현해도 될 것 같은?
bank["default"].sort_values().value_counts()

In [ ]:
bank["housing"].sort_values().value_counts()

In [ ]:
bank["loan"].sort_values().value_counts()

In [ ]:
bank["loan_all"] = 0
# yes = 1, no = 0로

In [ ]:
cond_loan = (bank["default"] == 'yes') | (bank["housing"] == 'yes') | (bank["loan"] == 'yes')

In [ ]:
bank.loc[(cond_loan),"loan_all"] = 1

In [ ]:
bank.head(6)

## 4. 라벨인코딩을 통해 모든 범주형 변수를 인코딩하라.

In [ ]:
#범주형변수: job	marital	education	default	housing	loan	contact	(month)	poutcome	y
from sklearn.preprocessing import LabelEncoder

labels = ["job", "marital", "education", "default", "housing", "loan", "contact", "poutcome", "y"]
for label in labels:
  le = LabelEncoder()
  le.fit(bank[label])
  bank[label] = le.transform(bank[label])
"""
le_job = LabelEncoder()
le_job.fit(bank["job"])
bank["job"] = le_job.transform(bank["job"])
"""
bank.head()

In [ ]:
le_marital = LabelEncoder()
le_marital.fit(bank["marital"])
bank["marital"] = le_marital.transform(bank["marital"])
bank.head()

In [ ]:
le_education = LabelEncoder()
le_education.fit(bank["education"])
bank["education"] = le_education.transform(bank["education"])
bank.head()

In [ ]:
le_default = LabelEncoder()
le_default.fit(bank["default"])
bank["default"] = le_default.transform(bank["default"])
bank.head()

In [ ]:
le_housing = LabelEncoder()
le_housing.fit(bank["housing"])
bank["housing"] = le_housing.transform(bank["housing"])
bank.head()

In [ ]:
le_loan = LabelEncoder()
le_loan.fit(bank["loan"])
bank["loan"] = le_loan.transform(bank["loan"])
bank.head()

In [ ]:
le_contact = LabelEncoder()
le_contact.fit(bank["contact"])
bank["contact"] = le_contact.transform(bank["contact"])
bank.head()

In [ ]:
le_poutcome = LabelEncoder()
le_poutcome.fit(bank["poutcome"])
bank["poutcome"] = le_poutcome.transform(bank["poutcome"])
bank.head()

In [ ]:
le_y = LabelEncoder()
le_y.fit(bank["y"])
bank["y"] = le_y.transform(bank["y"])
bank.head()

## 5. 쓸모없는 컬럼이 있다면 제거하고 그 근거를 제시하라.  

In [ ]:
# ID -> 그냥 사람 식별용이라 무의미함 // 월, 일은 지금 날짜를 모르는 상황에서 의미 있는 값으로 바꾸기 어려움.
bank.drop(columns = ["ID"], inplace=True)
bank.drop(columns = ["day", "month"], inplace=True)

In [ ]:
# 위에서 하나로 합쳤던 loan 관련 세가지 컬럼 모두 제거
bank.drop(columns = ["default", "housing", "loan"], inplace = True)

In [ ]:
bank.head()

## 6. train_test_split 메소드를 활용해 데이터를 학습 데이터와 검증 데이터로 나누어라. 

In [ ]:
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from xgboost import XGBClassifier #XGBoost
import lightgbm as lgb #LightGBM
from sklearn import tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
import graphviz 

In [ ]:
X = bank.drop(["y"], axis = 1)
y = bank["y"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 143)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## 7. 3가지 다른 모델로 데이터를 학습한 뒤 accuracy와 F1 score 차원에서 성능을 비교하라.

In [ ]:
#성능을 보여주는 함수
def evaluate_metrics(model, X_test, y_test):
  y_hat = model.predict(X_test)
  print("Accuracy: ", accuracy_score(y_test, y_hat), "\n")
  print("F1 score: ", f1_score(y_test, y_hat), "\n")

In [ ]:
#Logistic Regression
Log_R = LogisticRegression()
Log_R.fit(X_train, y_train)
evaluate_metrics(Log_R, X_test, y_test)

Accuracy:  0.7261072261072261 

F1 score:  0.32793136320305055 



In [ ]:
#Support Vector Machine - 너무오래걸림 ㅠㅠ
svc = svm.SVC(kernel = "linear", C = 0.1, gamma = 0.1)
svc.fit(X_train, y_train)
evaluate_metrics(svc, X_test, y_test)

In [ ]:
lgbClf = lgb.LGBMClassifier(num_leaves = 20)
lgbClf.fit(X_train, y_train)
evaluate_metrics(lgbClf, X_test, y_test)

In [ ]:
#RandomForest 학습
RfClf = RandomForestClassifier(n_estimators = 100, max_depth = 4)
RfClf.fit(X_train, y_train)
evaluate_metrics(RfClf, X_test, y_test)

## 8. 7번에서 가장 성능이 좋았던 모델을 Cross Validation해서 최종 accuracy와 F1 점수의 평균값을 제시하라.

In [ ]:
#Cross validation
from sklearn.model_selection import KFold
X = np.array(bank.drop(["y"], axis = 1))
y = bank["y"] 

kf = KFold(n_splits = 5, shuffle = True, random_state = 50)

In [ ]:
from sklearn.model_selection import cross_val_score #라이브러리 임포트
kf = KFold(n_splits = 5, shuffle = True, random_state = 50) #KFold 객체 생성

rfClf = RandomForestClassifier(n_estimators= 5, random_state = 0) #예시로 random forest 이용

cv_accuracy_result = cross_val_score(rfClf, X, y, cv=kf, scoring = "accuracy")
cv_f1_result = cross_val_score(rfClf, X, y, cv=kf, scoring = "f1")
print("accuracy mean: ", cv_accuracy_result.mean(), "\n")
print("f1 mean: ", cv_f1_result.mean(), "\n")